In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [7]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='ce222e1b-63fc-49f7-9814-ba0f3b74fd9d'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'The user asks: "Tell me about the langchain-mcp-adapters library". This is about LangChain? Possibly a library. Need to answer about it. Might need to search web. Use search_web.', 'tool_calls': [{'id': 'fc_f06658ca-3218-4c46-9c7b-b4e9923781ec', 'function': {'arguments': '{"query":"langchain-mcp-adapters library"}', 'name': 'search_web'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 266, 'total_tokens': 343, 'completion_time': 0.161122406, 'completion_tokens_details': {'reasoning_tokens': 44}, 'prompt_time': 0.014294422, 'prompt_tokens_details': None, 'queue_time': 0.005361632, 'total_time': 0.175416828}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_178e424650', 'ser

## Online MCP

In [9]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [10]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [11]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='5420e88c-77ff-4846-bd6f-dfeb9b6873cf'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks "What time is it?" No timezone given. We should assume local timezone is America/New_York per instructions. So we need to get current time in that timezone. Use get_current_time function.', 'tool_calls': [{'id': 'fc_758a847f-2810-43ff-baea-67ba1be61949', 'function': {'arguments': '{"timezone":"America/New_York"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 290, 'total_tokens': 364, 'completion_time': 0.153121666, 'completion_tokens_details': {'reasoning_tokens': 43}, 'prompt_time': 0.014401118, 'prompt_tokens_details': None, 'queue_time': 0.004796332, 'total_time': 0.167522784}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_309cec8e88', 'service_tier': 'on_de